In [3]:
# https://platform.openai.com/docs/guides/function-calling

In [21]:
import json
import openai


def generate_config(specification: dict) -> dict:
    print("Generating config...")
    example_specification = (
        {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "format": {
                            "type": "string",
                            "enum": ["celsius", "fahrenheit"],
                            "description": "The temperature unit to use. Infer this "
                            " from the users location.",
                        },
                    },
                    "required": ["location", "format"],
                },
            },
        },
    )

    example_config = {
        "name": "get_weather",
        "utterances": [
            "What is the weather like in SF?",
            "What is the weather in Cyprus?",
            "weather in London?",
            "Tell me the weather in New York",
            "what is the current weather in Paris?",
        ],
    }

    prompt = f"""
    Given the following specification, generate a config in a valid JSON format,
    Example:
    SPECIFICATION:
    {example_specification}

    CONFIG:
    {example_config}

    GIVEN SPECIFICATION:
    {specification}

    GENERATED CONFIG:
    """

    try:
        response = openai.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": f"{prompt}"},
            ],
        )
        ai_message = response.choices[0].message.content
        print("AI message:", ai_message)
        route_config = json.loads(ai_message)
        return route_config

    except json.JSONDecodeError as json_error:
        raise Exception("JSON parsing error", json_error)
    except Exception as e:
        raise Exception("Error generating config from Openai", e)

In [22]:
from semantic_router.schema import Route
from semantic_router.encoders import CohereEncoder
from semantic_router.layer import RouteLayer

def get_route_layer(config: list[dict]) -> RouteLayer:
    print("Getting route layer...")
    encoder = CohereEncoder()
    routes = [
        Route(name=route["name"], utterances=route["utterances"]) for route in config
    ]
    return RouteLayer(encoder=encoder, routes=routes)

In [25]:
specification = {
    "type": "function",
    "function": {
        "name": "get_time",
        "description": "Get the current time",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state",
                },
            },
            "required": ["location"],
        },
    },
}

route_config = generate_config(specification)
route_layer = get_route_layer([route_config])

queries = [
    "What is the weather like in Barcelona?",
    "What time is it in Taiwan?",
    "What is happening in the world?",
    "what is the time in Kaunas?",
    "Im bored",
    "I want to play a game",
    "Banana",
]

print("Getting function name for queries:\n")
for query in queries:
    function_name = route_layer(query)
    print((function_name, query))

Generating config...
AI message: {
    "name": "get_time",
    "utterances": [
        "What is the current time in SF?",
        "Tell me the time in London",
        "Could you tell me the time in New York?",
        "May I know the current time in Paris?",
        "Can you tell me what time is it in Singapore?"
    ]
}
Getting route layer...
Getting function name for queries:

(None, 'What is the weather like in Barcelona?')
('get_time', 'What time is it in Taiwan?')
(None, 'What is happening in the world?')
('get_time', 'what is the time in Kaunas?')
(None, 'Im bored')
(None, 'I want to play a game')
(None, 'Banana')
